In [7]:
# import sqlite3
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from keras import layers, models, optimizers
# from datetime import datetime
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service


In [2]:
# This is loading the input.
all_features_pd = pd.read_csv('mini-17430-NN.csv', sep=";", header=None)
all_features_array = np.array(all_features_pd)

# This is slicing out the bad rows (16348-16399) and the front 3k.
all_features_front = all_features_array[:16348,:] 
all_features_back = all_features_array[16400:,:]

all_features_array = np.vstack((all_features_front, all_features_back))

# This is attempting to clear memory
all_features_pd = []
del all_features_pd
all_features_front = []
all_features_back = []
del all_features_front
del all_features_back

print(all_features_array.shape)
num_features = len(all_features_array)

coach1 = all_features_array[:,0:60]
player1 = all_features_array[:,60:637]
player2 = all_features_array[:,637:1214]
player3 = all_features_array[:,1214:1791]
player4 = all_features_array[:,1791:2368]
player5 = all_features_array[:,2368:2945]
player6 = all_features_array[:,2945:3522]
player7 = all_features_array[:,3522:4099]
player8 = all_features_array[:,4099:4676]
player9 = all_features_array[:,4676:5253]
player10 = all_features_array[:,5253:5830]
player11 = all_features_array[:,5830:6407]
player12 = all_features_array[:,6407:6984]
player13 = all_features_array[:,6984:7561]
player14 = all_features_array[:,7561:8138]
player15 = all_features_array[:,8138:8715]
coach2 = all_features_array[:,8715:8775]
player16 = all_features_array[:,8775:9352]
player17 = all_features_array[:,9352:9929]
player18 = all_features_array[:,9929:10506]
player19 = all_features_array[:,10506:11083]
player20 = all_features_array[:,11083:11660]
player21 = all_features_array[:,11660:12237]
player22 = all_features_array[:,12237:12814]
player23 = all_features_array[:,12814:13391]
player24 = all_features_array[:,13391:13968]
player25 = all_features_array[:,13968:14545]
player26 = all_features_array[:,14545:15122]
player27 = all_features_array[:,15122:15699]
player28 = all_features_array[:,15699:16276]
player29 = all_features_array[:,16276:16853]
player30 = all_features_array[:,16853:17430]

input_features = np.array([player1, player2, player3, player4, player5, player6, player7, player8, player9
                         , player10, player11, player12, player13, player14, player15, player16, player17
                         , player18, player19, player20, player21, player22, player23, player24, player25
                         , player26, player27, player28, player29, player30])

# Another attempt to free memory:
all_features_array = []
del all_features_array


(17455, 17430)


In [3]:
# This is loading the point-based labels.
all_labels_pd = pd.read_csv('mini-DK-scoresheet.csv', sep=";", header=None)
all_labels_array = np.array(all_labels_pd)

# This is slicing out the bad rows (16348-16399) and the front 3k.
all_labels_front = all_labels_array[:16348,:]
all_labels_back = all_labels_array[16400:,:]

all_labels_array = np.vstack((all_labels_front, all_labels_back))

# This is attempting to clear memory
all_labels_pd = []
del all_labels_pd
all_labels_front = []
all_labels_back = []
del all_labels_front
del all_labels_back

print(all_labels_array.shape)

score1 = all_labels_array[:,0].reshape(num_features,1)
score2 = all_labels_array[:,1].reshape(num_features,1)
score3 = all_labels_array[:,2].reshape(num_features,1)
score4 = all_labels_array[:,3].reshape(num_features,1)
score5 = all_labels_array[:,4].reshape(num_features,1)
score6 = all_labels_array[:,5].reshape(num_features,1)
score7 = all_labels_array[:,6].reshape(num_features,1)
score8 = all_labels_array[:,7].reshape(num_features,1)
score9 = all_labels_array[:,8].reshape(num_features,1)
score10 = all_labels_array[:,9].reshape(num_features,1)
score11 = all_labels_array[:,10].reshape(num_features,1)
score12 = all_labels_array[:,11].reshape(num_features,1)
score13 = all_labels_array[:,12].reshape(num_features,1)
score14 = all_labels_array[:,13].reshape(num_features,1)
score15 = all_labels_array[:,14].reshape(num_features,1)
score16 = all_labels_array[:,15].reshape(num_features,1)
score17 = all_labels_array[:,16].reshape(num_features,1)
score18 = all_labels_array[:,17].reshape(num_features,1)
score19 = all_labels_array[:,18].reshape(num_features,1)
score20 = all_labels_array[:,19].reshape(num_features,1)
score21 = all_labels_array[:,20].reshape(num_features,1)
score22 = all_labels_array[:,21].reshape(num_features,1)
score23 = all_labels_array[:,22].reshape(num_features,1)
score24 = all_labels_array[:,23].reshape(num_features,1)
score25 = all_labels_array[:,24].reshape(num_features,1)
score26 = all_labels_array[:,25].reshape(num_features,1)
score27 = all_labels_array[:,26].reshape(num_features,1)
score28 = all_labels_array[:,27].reshape(num_features,1)
score29 = all_labels_array[:,28].reshape(num_features,1)
score30 = all_labels_array[:,29].reshape(num_features,1)

output_scores = np.array([score1, score2, score3, score4, score5, score6, score7, score8, score9, score10, score11, 
                        score12, score13, score14, score15, score16, score17, score18, score19, score20
                       , score21, score22, score23, score24, score25, score26, score27, score28, score29, score30])

# Another attempt to free memory:
all_labels_array = []
del all_labels_array


(17455, 30)


In [4]:
def super_shuffle(player_start, score_start, coach1, coach2, start, stop):
    away_team_shuffle = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
    home_team_shuffle = np.array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

    np.random.shuffle(away_team_shuffle)
    np.random.shuffle(home_team_shuffle)

    shuffled_features_array = np.hstack((coach1[start:stop,:], player_start[away_team_shuffle[0]][start:stop,:],  
                  player_start[away_team_shuffle[1]][start:stop,:], player_start[away_team_shuffle[2]][start:stop,:], 
                  player_start[away_team_shuffle[3]][start:stop,:],
                  player_start[away_team_shuffle[4]][start:stop,:], player_start[away_team_shuffle[5]][start:stop,:],
                  player_start[away_team_shuffle[6]][start:stop,:], player_start[away_team_shuffle[7]][start:stop,:],
                  player_start[away_team_shuffle[8]][start:stop,:], player_start[away_team_shuffle[9]][start:stop,:],
                  player_start[away_team_shuffle[10]][start:stop,:], player_start[away_team_shuffle[11]][start:stop,:],
                  player_start[away_team_shuffle[12]][start:stop,:], player_start[away_team_shuffle[13]][start:stop,:],
                  player_start[away_team_shuffle[14]][start:stop,:], coach2[start:stop,:],  
                  player_start[home_team_shuffle[0]][start:stop,:],
                  player_start[home_team_shuffle[1]][start:stop,:], player_start[home_team_shuffle[2]][start:stop,:],
                  player_start[home_team_shuffle[3]][start:stop,:], player_start[home_team_shuffle[4]][start:stop,:],
                  player_start[home_team_shuffle[5]][start:stop,:], player_start[home_team_shuffle[6]][start:stop,:],
                  player_start[home_team_shuffle[7]][start:stop,:], player_start[home_team_shuffle[8]][start:stop,:],
                  player_start[home_team_shuffle[9]][start:stop,:], player_start[home_team_shuffle[10]][start:stop,:],
                  player_start[home_team_shuffle[11]][start:stop,:], player_start[home_team_shuffle[12]][start:stop,:],
                  player_start[home_team_shuffle[13]][start:stop,:], player_start[home_team_shuffle[14]][start:stop,:]))
    
    shuffled_labels_array = np.hstack((score_start[away_team_shuffle[0]][start:stop,:],  
                                       score_start[away_team_shuffle[1]][start:stop,:],
                  score_start[away_team_shuffle[2]][start:stop,:], score_start[away_team_shuffle[3]][start:stop,:],
                  score_start[away_team_shuffle[4]][start:stop,:], score_start[away_team_shuffle[5]][start:stop,:],
                  score_start[away_team_shuffle[6]][start:stop,:], score_start[away_team_shuffle[7]][start:stop,:],
                  score_start[away_team_shuffle[8]][start:stop,:], score_start[away_team_shuffle[9]][start:stop,:],
                  score_start[away_team_shuffle[10]][start:stop,:], score_start[away_team_shuffle[11]][start:stop,:],
                  score_start[away_team_shuffle[12]][start:stop,:], score_start[away_team_shuffle[13]][start:stop,:],
                  score_start[away_team_shuffle[14]][start:stop,:], score_start[home_team_shuffle[0]][start:stop,:],
                  score_start[home_team_shuffle[1]][start:stop,:], score_start[home_team_shuffle[2]][start:stop,:],
                  score_start[home_team_shuffle[3]][start:stop,:], score_start[home_team_shuffle[4]][start:stop,:],
                  score_start[home_team_shuffle[5]][start:stop,:], score_start[home_team_shuffle[6]][start:stop,:],
                  score_start[home_team_shuffle[7]][start:stop,:], score_start[home_team_shuffle[8]][start:stop,:],
                  score_start[home_team_shuffle[9]][start:stop,:], score_start[home_team_shuffle[10]][start:stop,:],
                  score_start[home_team_shuffle[11]][start:stop,:], score_start[home_team_shuffle[12]][start:stop,:],
                  score_start[home_team_shuffle[13]][start:stop,:], score_start[home_team_shuffle[14]][start:stop,:]))   

    return shuffled_features_array, shuffled_labels_array


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!CONTENT BELOW THIS LINE IS REGRESSION TF FOR 30 OUTPUTS !!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [5]:
# This will initialize a new model:
model = models.Sequential()
model.add(layers.Dense(17430, activation='relu', input_dim=17430))
model.add(layers.Dense(17430, activation='relu'))
model.add(layers.Dense(30))


2023-04-04 13:29:29.223042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
# Or this will load an existing model:
model = tf.keras.models.load_model("thirty_player.model", compile=False)


In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=.001,
    decay_steps=100,
    decay_rate=0.95)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=tf.keras.metrics.MeanAbsoluteError())


In [8]:
# This is putting the whole thing in a loop to it can be run a bunch of times
for repeats in range(10):
    chunk_size = 100
    i = 0

    start_features, start_labels = super_shuffle(input_features, output_scores, coach1, coach2, i, i+chunk_size)
    combined_features = np.hstack((start_features, start_labels))

    while i < num_features:
        i += chunk_size # This comes first because the initialization is above.
        for copies in range(1): # This multiplies the dataset.
            new_features, new_labels = super_shuffle(input_features, output_scores,coach1, coach2, i, i+chunk_size)
            new_combined_features = np.hstack((new_features, new_labels))
            combined_features = np.vstack((new_combined_features, combined_features))

    # Freeing memory
    start_features = []
    del start_features
    start_labels = []
    del start_labels
    new_features = []
    del new_features
    new_labels = []
    del new_labels

    combined_features = tf.random.shuffle(combined_features)
    full_team_inputs = combined_features[:,:17430]
    full_team_scores = combined_features[:,17430:17461]

    testing_inputs = full_team_inputs[:500,:]
    testing_scores = full_team_scores[:500,:]

    # Freeing memory:
    combined_features = []
    del combined_features

    model.evaluate(testing_inputs, testing_scores)
    model.fit(full_team_inputs, full_team_scores, epochs=1)
    

546/546 [==============================] - 1080s 2s/step - loss: 103.0773 - mean_absolute_error: 7.3250


In [9]:
model.save('thirty_player.model')  


INFO:tensorflow:Assets written to: thirty_player.model/assets


INFO:tensorflow:Assets written to: thirty_player.model/assets


In [11]:
print(full_team_inputs.shape)
print(full_team_scores.shape)

print(full_team_inputs)
print(full_team_scores)


(17455, 17430)
(17455, 30)
tf.Tensor(
[[ 1.          0.          1.         ...  0.06666667  0.06666667
   0.06666667]
 [ 1.          1.          0.         ...  0.03333333  0.
  -0.01666667]
 [ 1.          1.          1.         ...  0.          0.
   0.        ]
 ...
 [ 1.          1.          1.         ...  0.          0.
   0.        ]
 [ 1.          1.          0.         ...  0.          0.
   0.        ]
 [ 1.          0.          0.         ...  0.03333333  0.06666667
   0.06666667]], shape=(17455, 17430), dtype=float64)
tf.Tensor(
[[11.25  3.    8.   ...  0.   13.   32.75]
 [ 4.5  21.25  0.   ...  0.    0.    0.  ]
 [12.75 13.5  16.75 ...  0.   32.25 13.25]
 ...
 [23.75  0.    9.75 ...  0.   40.    0.  ]
 [ 0.    0.   21.75 ... 37.75 10.75  0.  ]
 [ 0.   24.5   6.75 ...  3.5   0.    7.5 ]], shape=(17455, 30), dtype=float64)


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !! THIS IS ALL THE BINOMIAL MODEL STUFF!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [12]:
# Initilizing a new model:
model = models.Sequential()
model.add(layers.Dense(17430, activation='relu', input_dim=17430))
model.add(layers.Dense(17430, activation='relu'))
model.add(layers.Dense(30, activation='sigmoid'))


In [ ]:
# Or loading an existing model.
model = tf.keras.models.load_model("binomial.model", compile=False)


In [13]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=.00001,
    decay_steps=80,
    decay_rate=0.95)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=tf.keras.metrics.BinaryAccuracy())


In [15]:
# This is putting the whole thing in a loop so it can be run a bunch of times
# !! Don't forget to load the super_shuffle function before running !!

# This is putting the whole thing in a loop to it can be run a bunch of times
for repeats in range(10):
    chunk_size = 100
    i = 0

    start_features, start_labels = super_shuffle(input_features, output_scores, coach1, coach2, i, i+chunk_size)
    combined_features = np.hstack((start_features, start_labels))

    while i < num_features:
        i += chunk_size # This comes first because the initialization is above.
        for copies in range(1): # This multiplies the dataset.
            new_features, new_labels = super_shuffle(input_features, output_scores,coach1, coach2, i, i+chunk_size)
            new_combined_features = np.hstack((new_features, new_labels))
            combined_features = np.vstack((new_combined_features, combined_features))

    # Freeing memory
    start_features = []
    del start_features
    start_labels = []
    del start_labels
    new_features = []
    del new_features
    new_labels = []
    del new_labels

    combined_features = tf.random.shuffle(combined_features)
    full_team_inputs = combined_features[:,:17430]    
    
    binomial_scores = []
    for line in combined_features[:,17430:17461]:
        new_line = []
        for data_point in line:
            if data_point > 40:
                new_line.append(1)
            else:
                new_line.append(0)
        binomial_scores.append(new_line)
    
    binomial_scores = tf.constant(binomial_scores)
    testing_inputs = full_team_inputs[:500,:]    
    binomial_testing_scores = binomial_scores[:500,:]
    
    # Freeing memory:
    combined_features = []
    del combined_features

    model.evaluate(testing_inputs, binomial_testing_scores)
    model.fit(full_team_inputs, binomial_scores, epochs=1)
    

255/546 [=============>................] - ETA: 9:54 - loss: 0.2048 - binary_accuracy: 0.9169

KeyboardInterrupt: 

In [16]:
model.save('binomial.model')

INFO:tensorflow:Assets written to: binomial.model/assets


INFO:tensorflow:Assets written to: binomial.model/assets


In [ ]:
# This is testing the quailty of the output:
x = 3833
for i in range(10):
    test_input = np.reshape(full_team_inputs[x+i], (1,17430))
    network_output = model.predict(test_input, verbose=0)
    print("Sample number: " + str(3833+i))
    for j in range(30):
        print("Result: " + str(full_team_scores[x+i][j]) + " vs prediction: " + str(network_output[0][j]))


In [20]:
import sys
!{sys.executable} -m pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.3 MB/s eta 0:00:0000:0100:01


In [8]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !! THIS IS THE START OF THE LightGBM SECTION!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

import optuna
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split


In [33]:
# Preparing data.
chunk_size = 100
i = 0

start_features, start_labels = super_shuffle(input_features, output_scores, coach1, coach2, i, i+chunk_size)
combined_features = np.hstack((start_features, start_labels))

while i < num_features:
    i += chunk_size # This comes first because the initialization is above.
    for copies in range(1): # This multiplies the dataset.
        new_features, new_labels = super_shuffle(input_features, output_scores, coach1, coach2, i, i+chunk_size)
        new_combined_features = np.hstack((new_features, new_labels))
        combined_features = np.vstack((new_combined_features, combined_features))

# Freeing memory
start_features = []
del start_features
start_labels = []
del start_labels
new_features = []
del new_features
new_labels = []
del new_labels

np.random.shuffle(combined_features)

all_features_array = combined_features[:,:17430]

train_x = all_features_array[:14000,:]
valid_x = all_features_array[14000:,:]

all_features_array = []
del all_features_array



In [34]:
# This builds the REGRESSION labels for a single player at a time.
# Starting with the away team:

away_player_training_list = []
away_player_validation_list = []

for i in range(15):
    player = combined_features[:,(17430+i):(17431+i)]
    player_train = player[:14000,:]
    player_test = player[14000:,:]
    player_train = player_train.ravel()
    player_test = player_test.ravel()
    away_player_training_list.append(player_train)
    away_player_validation_list.append(player_test)

# Then the home team:
home_player_training_list = []
home_player_validation_list = []

for i in range(15):
    player = combined_features[:,17445+i:17446+i]
    player_train = player[:14000,:]
    player_test = player[14000:,:]
    player_train = player_train.ravel()
    player_test = player_test.ravel()
    home_player_training_list.append(player_train)
    home_player_validation_list.append(player_test)

# Cleaning up variables:
combined_features = []
all_labels_away = []
all_labels_home = []
del all_labels_away
del all_labels_home
del combined_features


In [31]:
# This section trains 15 models on (hopefully) optimized parameters.
away_model_list = ['player1_away', 'player2_away', 'player3_away', 'player4_away', 'player5_away', 
                  'player6_away', 'player7_away', 'player8_away', 'player9_away', 'player10_away', 
                  'player11_away', 'player12_away', 'player13_away', 'player14_away', 'player15_away']

for j in range(15):
    train_y = away_player_training_list[j]
    valid_y = away_player_validation_list[j]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "regression",
        "metric": ['l1','l2'],
        "force_col_wise": True,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": .5,
        "lambda_l2": .5,
        "num_leaves": 1000,
        "bagging_fraction": .9,
        "bagging_freq": 3,
        "min_child_samples": 100,
        'learning_rate': .01,
        'n_estimators': 600,
        'max_depth': 30,
        'max_bin': 400,
    }

    gbm = lgb.train(param, dtrain, valid_sets=[dvalid])
    model_name = away_model_list[j] + '.txt'
    gbm.save_model(model_name)


[1]	valid_0's l1: 13.3623	valid_0's l2: 265.182
[2]	valid_0's l1: 13.2701	valid_0's l2: 261.719
[3]	valid_0's l1: 13.1786	valid_0's l2: 258.326
[4]	valid_0's l1: 13.0874	valid_0's l2: 254.958
[5]	valid_0's l1: 12.997	valid_0's l2: 251.65
[6]	valid_0's l1: 12.909	valid_0's l2: 248.434
[7]	valid_0's l1: 12.8217	valid_0's l2: 245.285
[8]	valid_0's l1: 12.7358	valid_0's l2: 242.19
[9]	valid_0's l1: 12.6517	valid_0's l2: 239.153
[10]	valid_0's l1: 12.569	valid_0's l2: 236.17
[11]	valid_0's l1: 12.4867	valid_0's l2: 233.228
[12]	valid_0's l1: 12.4044	valid_0's l2: 230.324
[13]	valid_0's l1: 12.3251	valid_0's l2: 227.527
[14]	valid_0's l1: 12.2464	valid_0's l2: 224.754
[15]	valid_0's l1: 12.1675	valid_0's l2: 222.033
[16]	valid_0's l1: 12.0902	valid_0's l2: 219.348
[17]	valid_0's l1: 12.0149	valid_0's l2: 216.801
[18]	valid_0's l1: 11.9396	valid_0's l2: 214.249
[19]	valid_0's l1: 11.8648	valid_0's l2: 211.742
[20]	valid_0's l1: 11.7915	valid_0's l2: 209.291
[21]	valid_0's l1: 11.7187	valid_0'

[169]	valid_0's l1: 6.83821	valid_0's l2: 88.0847
[170]	valid_0's l1: 6.82675	valid_0's l2: 87.9287
[171]	valid_0's l1: 6.81535	valid_0's l2: 87.7685
[172]	valid_0's l1: 6.80477	valid_0's l2: 87.6359
[173]	valid_0's l1: 6.79464	valid_0's l2: 87.5072
[174]	valid_0's l1: 6.78462	valid_0's l2: 87.3813
[175]	valid_0's l1: 6.77325	valid_0's l2: 87.2364
[176]	valid_0's l1: 6.76245	valid_0's l2: 87.1018
[177]	valid_0's l1: 6.75182	valid_0's l2: 86.9597
[178]	valid_0's l1: 6.74146	valid_0's l2: 86.8317
[179]	valid_0's l1: 6.73057	valid_0's l2: 86.6861
[180]	valid_0's l1: 6.72085	valid_0's l2: 86.5654
[181]	valid_0's l1: 6.71096	valid_0's l2: 86.4487
[182]	valid_0's l1: 6.70128	valid_0's l2: 86.3268
[183]	valid_0's l1: 6.69159	valid_0's l2: 86.1991
[184]	valid_0's l1: 6.68185	valid_0's l2: 86.0909
[185]	valid_0's l1: 6.67219	valid_0's l2: 85.985
[186]	valid_0's l1: 6.66155	valid_0's l2: 85.8478
[187]	valid_0's l1: 6.65279	valid_0's l2: 85.7366
[188]	valid_0's l1: 6.64325	valid_0's l2: 85.6275
[

[334]	valid_0's l1: 5.95623	valid_0's l2: 78.7707
[335]	valid_0's l1: 5.9541	valid_0's l2: 78.7535
[336]	valid_0's l1: 5.95156	valid_0's l2: 78.7329
[337]	valid_0's l1: 5.95041	valid_0's l2: 78.7332
[338]	valid_0's l1: 5.94751	valid_0's l2: 78.713
[339]	valid_0's l1: 5.94498	valid_0's l2: 78.6983
[340]	valid_0's l1: 5.94375	valid_0's l2: 78.6881
[341]	valid_0's l1: 5.94256	valid_0's l2: 78.6797
[342]	valid_0's l1: 5.94003	valid_0's l2: 78.6673
[343]	valid_0's l1: 5.93769	valid_0's l2: 78.6439
[344]	valid_0's l1: 5.93634	valid_0's l2: 78.6325
[345]	valid_0's l1: 5.93496	valid_0's l2: 78.6196
[346]	valid_0's l1: 5.93262	valid_0's l2: 78.6081
[347]	valid_0's l1: 5.93083	valid_0's l2: 78.5967
[348]	valid_0's l1: 5.92985	valid_0's l2: 78.5984
[349]	valid_0's l1: 5.92842	valid_0's l2: 78.5881
[350]	valid_0's l1: 5.92731	valid_0's l2: 78.5819
[351]	valid_0's l1: 5.92622	valid_0's l2: 78.5743
[352]	valid_0's l1: 5.92327	valid_0's l2: 78.5299
[353]	valid_0's l1: 5.92092	valid_0's l2: 78.5227
[3

[499]	valid_0's l1: 5.78307	valid_0's l2: 76.9365
[500]	valid_0's l1: 5.78208	valid_0's l2: 76.9146
[501]	valid_0's l1: 5.78115	valid_0's l2: 76.8957
[502]	valid_0's l1: 5.78108	valid_0's l2: 76.8996
[503]	valid_0's l1: 5.78081	valid_0's l2: 76.8997
[504]	valid_0's l1: 5.77984	valid_0's l2: 76.8894
[505]	valid_0's l1: 5.77956	valid_0's l2: 76.8816
[506]	valid_0's l1: 5.77917	valid_0's l2: 76.8799
[507]	valid_0's l1: 5.77881	valid_0's l2: 76.8739
[508]	valid_0's l1: 5.77789	valid_0's l2: 76.8637
[509]	valid_0's l1: 5.77708	valid_0's l2: 76.8591
[510]	valid_0's l1: 5.77711	valid_0's l2: 76.8598
[511]	valid_0's l1: 5.77638	valid_0's l2: 76.8459
[512]	valid_0's l1: 5.77619	valid_0's l2: 76.8407
[513]	valid_0's l1: 5.77595	valid_0's l2: 76.8301
[514]	valid_0's l1: 5.77543	valid_0's l2: 76.8276
[515]	valid_0's l1: 5.77496	valid_0's l2: 76.8205
[516]	valid_0's l1: 5.77504	valid_0's l2: 76.8213
[517]	valid_0's l1: 5.77409	valid_0's l2: 76.8147
[518]	valid_0's l1: 5.77339	valid_0's l2: 76.8067


[65]	valid_0's l1: 9.42235	valid_0's l2: 144.032
[66]	valid_0's l1: 9.38115	valid_0's l2: 143.001
[67]	valid_0's l1: 9.33945	valid_0's l2: 141.985
[68]	valid_0's l1: 9.29861	valid_0's l2: 140.993
[69]	valid_0's l1: 9.25741	valid_0's l2: 139.992
[70]	valid_0's l1: 9.21718	valid_0's l2: 139.064
[71]	valid_0's l1: 9.17852	valid_0's l2: 138.141
[72]	valid_0's l1: 9.14062	valid_0's l2: 137.238
[73]	valid_0's l1: 9.10111	valid_0's l2: 136.293
[74]	valid_0's l1: 9.06428	valid_0's l2: 135.423
[75]	valid_0's l1: 9.02771	valid_0's l2: 134.568
[76]	valid_0's l1: 8.99309	valid_0's l2: 133.783
[77]	valid_0's l1: 8.9589	valid_0's l2: 133.017
[78]	valid_0's l1: 8.92308	valid_0's l2: 132.206
[79]	valid_0's l1: 8.88836	valid_0's l2: 131.4
[80]	valid_0's l1: 8.85394	valid_0's l2: 130.605
[81]	valid_0's l1: 8.81986	valid_0's l2: 129.825
[82]	valid_0's l1: 8.78665	valid_0's l2: 129.051
[83]	valid_0's l1: 8.75372	valid_0's l2: 128.313
[84]	valid_0's l1: 8.72055	valid_0's l2: 127.578
[85]	valid_0's l1: 8.68

...

[482]	valid_0's l1: 6.05411	valid_0's l2: 86.5208
[483]	valid_0's l1: 6.05392	valid_0's l2: 86.516
[484]	valid_0's l1: 6.0543	valid_0's l2: 86.5244
[485]	valid_0's l1: 6.0537	valid_0's l2: 86.5112
[486]	valid_0's l1: 6.0532	valid_0's l2: 86.503
[487]	valid_0's l1: 6.05271	valid_0's l2: 86.497
[488]	valid_0's l1: 6.05197	valid_0's l2: 86.4814
[489]	valid_0's l1: 6.05158	valid_0's l2: 86.4617
[490]	valid_0's l1: 6.05127	valid_0's l2: 86.4531
[491]	valid_0's l1: 6.05043	valid_0's l2: 86.4403
[492]	valid_0's l1: 6.04954	valid_0's l2: 86.4208
[493]	valid_0's l1: 6.04898	valid_0's l2: 86.4057
[494]	valid_0's l1: 6.04825	valid_0's l2: 86.3952
[495]	valid_0's l1: 6.04806	valid_0's l2: 86.3889
[496]	valid_0's l1: 6.04848	valid_0's l2: 86.403
[497]	valid_0's l1: 6.04867	valid_0's l2: 86.4159
[498]	valid_0's l1: 6.04855	valid_0's l2: 86.413
[499]	valid_0's l1: 6.04826	valid_0's l2: 86.4136
[500]	valid_0's l1: 6.04814	valid_0's l2: 86.4161
[501]	valid_0's l1: 6.04796	valid_0's l2: 86.4151
[502]	va

[48]	valid_0's l1: 10.6301	valid_0's l2: 172.382
[49]	valid_0's l1: 10.5809	valid_0's l2: 170.942
[50]	valid_0's l1: 10.5325	valid_0's l2: 169.533
[51]	valid_0's l1: 10.4849	valid_0's l2: 168.162
[52]	valid_0's l1: 10.4369	valid_0's l2: 166.795
[53]	valid_0's l1: 10.3896	valid_0's l2: 165.451
[54]	valid_0's l1: 10.3428	valid_0's l2: 164.133
[55]	valid_0's l1: 10.2966	valid_0's l2: 162.851
[56]	valid_0's l1: 10.2513	valid_0's l2: 161.596
[57]	valid_0's l1: 10.206	valid_0's l2: 160.361
[58]	valid_0's l1: 10.1638	valid_0's l2: 159.209
[59]	valid_0's l1: 10.1208	valid_0's l2: 158.023
[60]	valid_0's l1: 10.0794	valid_0's l2: 156.895
[61]	valid_0's l1: 10.0385	valid_0's l2: 155.779
[62]	valid_0's l1: 9.99844	valid_0's l2: 154.712
[63]	valid_0's l1: 9.95867	valid_0's l2: 153.65
[64]	valid_0's l1: 9.92081	valid_0's l2: 152.645
[65]	valid_0's l1: 9.88095	valid_0's l2: 151.575
[66]	valid_0's l1: 9.84367	valid_0's l2: 150.611
[67]	valid_0's l1: 9.80375	valid_0's l2: 149.596
[68]	valid_0's l1: 9.7

[214]	valid_0's l1: 7.20846	valid_0's l2: 96.8549
[215]	valid_0's l1: 7.20113	valid_0's l2: 96.7583
[216]	valid_0's l1: 7.19447	valid_0's l2: 96.6604
[217]	valid_0's l1: 7.18696	valid_0's l2: 96.5454
[218]	valid_0's l1: 7.17964	valid_0's l2: 96.4378
[219]	valid_0's l1: 7.17187	valid_0's l2: 96.3172
[220]	valid_0's l1: 7.1662	valid_0's l2: 96.2452
[221]	valid_0's l1: 7.1597	valid_0's l2: 96.1565
[222]	valid_0's l1: 7.15247	valid_0's l2: 96.0491
[223]	valid_0's l1: 7.14561	valid_0's l2: 95.9453
[224]	valid_0's l1: 7.13952	valid_0's l2: 95.8676
[225]	valid_0's l1: 7.13352	valid_0's l2: 95.7945
[226]	valid_0's l1: 7.12638	valid_0's l2: 95.6944
[227]	valid_0's l1: 7.11966	valid_0's l2: 95.6038
[228]	valid_0's l1: 7.11375	valid_0's l2: 95.5241
[229]	valid_0's l1: 7.10835	valid_0's l2: 95.4449
[230]	valid_0's l1: 7.10339	valid_0's l2: 95.3725
[231]	valid_0's l1: 7.09789	valid_0's l2: 95.2903
[232]	valid_0's l1: 7.09154	valid_0's l2: 95.211
[233]	valid_0's l1: 7.08515	valid_0's l2: 95.1417
[23

[379]	valid_0's l1: 6.64251	valid_0's l2: 90.1018
[380]	valid_0's l1: 6.64124	valid_0's l2: 90.0858
[381]	valid_0's l1: 6.64056	valid_0's l2: 90.085
[382]	valid_0's l1: 6.63989	valid_0's l2: 90.0792
[383]	valid_0's l1: 6.63888	valid_0's l2: 90.0704
[384]	valid_0's l1: 6.63727	valid_0's l2: 90.0524
[385]	valid_0's l1: 6.63671	valid_0's l2: 90.0506
[386]	valid_0's l1: 6.63685	valid_0's l2: 90.0637
[387]	valid_0's l1: 6.63666	valid_0's l2: 90.0704
[388]	valid_0's l1: 6.63556	valid_0's l2: 90.0624
[389]	valid_0's l1: 6.63519	valid_0's l2: 90.0543
[390]	valid_0's l1: 6.63469	valid_0's l2: 90.056
[391]	valid_0's l1: 6.63299	valid_0's l2: 90.0301
[392]	valid_0's l1: 6.63095	valid_0's l2: 90.0071
[393]	valid_0's l1: 6.62976	valid_0's l2: 89.9955
[394]	valid_0's l1: 6.6288	valid_0's l2: 89.9847
[395]	valid_0's l1: 6.62769	valid_0's l2: 89.9697
[396]	valid_0's l1: 6.62657	valid_0's l2: 89.9671
[397]	valid_0's l1: 6.62569	valid_0's l2: 89.9449
[398]	valid_0's l1: 6.62464	valid_0's l2: 89.9308
[39

[544]	valid_0's l1: 6.51805	valid_0's l2: 88.3409
[545]	valid_0's l1: 6.51743	valid_0's l2: 88.3243
[546]	valid_0's l1: 6.51704	valid_0's l2: 88.3151
[547]	valid_0's l1: 6.51661	valid_0's l2: 88.3047
[548]	valid_0's l1: 6.51611	valid_0's l2: 88.2946
[549]	valid_0's l1: 6.5151	valid_0's l2: 88.281
[550]	valid_0's l1: 6.51474	valid_0's l2: 88.2789
[551]	valid_0's l1: 6.51484	valid_0's l2: 88.2784
[552]	valid_0's l1: 6.5145	valid_0's l2: 88.2693
[553]	valid_0's l1: 6.51463	valid_0's l2: 88.2705
[554]	valid_0's l1: 6.51414	valid_0's l2: 88.2644
[555]	valid_0's l1: 6.51357	valid_0's l2: 88.2523
[556]	valid_0's l1: 6.51229	valid_0's l2: 88.2414
[557]	valid_0's l1: 6.5121	valid_0's l2: 88.2455
[558]	valid_0's l1: 6.51116	valid_0's l2: 88.2372
[559]	valid_0's l1: 6.51104	valid_0's l2: 88.2281
[560]	valid_0's l1: 6.51035	valid_0's l2: 88.2182
[561]	valid_0's l1: 6.51011	valid_0's l2: 88.2188
[562]	valid_0's l1: 6.50925	valid_0's l2: 88.2157
[563]	valid_0's l1: 6.509	valid_0's l2: 88.2176
[564]	

In [35]:
# This section trains 15 models on (hopefully) optimized parameters.
home_model_list = ['player1_home', 'player2_home', 'player3_home', 'player4_home', 'player5_home', 
                  'player6_home', 'player7_home', 'player8_home', 'player9_home', 'player10_home', 
                  'player11_home', 'player12_home', 'player13_home', 'player14_home', 'player15_home']

for j in range(15):
    train_y = home_player_training_list[j]
    valid_y = home_player_validation_list[j]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "regression",
        "metric": ['l1','l2'],
        "force_col_wise": True,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": .5,
        "lambda_l2": .5,
        "num_leaves": 1000,
        "bagging_fraction": .9,
        "bagging_freq": 3,
        "min_child_samples": 100,
        'learning_rate': .01,
        'n_estimators': 600,
        'max_depth': 30,
        'max_bin': 400,
    }

    gbm = lgb.train(param, dtrain, valid_sets=[dvalid])
    model_name = home_model_list[j] + '.txt'
    gbm.save_model(model_name)
    print("Completed model: " + str(j+1))


[1]	valid_0's l1: 13.8498	valid_0's l2: 283.232
[2]	valid_0's l1: 13.7597	valid_0's l2: 279.774
[3]	valid_0's l1: 13.6706	valid_0's l2: 276.36
[4]	valid_0's l1: 13.5834	valid_0's l2: 273.05
[5]	valid_0's l1: 13.4947	valid_0's l2: 269.655
[6]	valid_0's l1: 13.4104	valid_0's l2: 266.485
[7]	valid_0's l1: 13.3242	valid_0's l2: 263.264
[8]	valid_0's l1: 13.241	valid_0's l2: 260.185
[9]	valid_0's l1: 13.1566	valid_0's l2: 257.066
[10]	valid_0's l1: 13.0739	valid_0's l2: 254.044
[11]	valid_0's l1: 12.9926	valid_0's l2: 251.117
[12]	valid_0's l1: 12.9119	valid_0's l2: 248.219
[13]	valid_0's l1: 12.8316	valid_0's l2: 245.333
[14]	valid_0's l1: 12.7524	valid_0's l2: 242.509
[15]	valid_0's l1: 12.6734	valid_0's l2: 239.752
[16]	valid_0's l1: 12.5989	valid_0's l2: 237.145
[17]	valid_0's l1: 12.5259	valid_0's l2: 234.61
[18]	valid_0's l1: 12.4517	valid_0's l2: 232.074
[19]	valid_0's l1: 12.3781	valid_0's l2: 229.511
[20]	valid_0's l1: 12.3072	valid_0's l2: 227.084
[21]	valid_0's l1: 12.2354	valid_

[168]	valid_0's l1: 7.38001	valid_0's l2: 99.0668
[169]	valid_0's l1: 7.36726	valid_0's l2: 98.8555
[170]	valid_0's l1: 7.35555	valid_0's l2: 98.6613
[171]	valid_0's l1: 7.34378	valid_0's l2: 98.461
[172]	valid_0's l1: 7.3317	valid_0's l2: 98.2635
[173]	valid_0's l1: 7.3194	valid_0's l2: 98.0627
[174]	valid_0's l1: 7.30785	valid_0's l2: 97.8807
[175]	valid_0's l1: 7.29541	valid_0's l2: 97.6659
[176]	valid_0's l1: 7.28362	valid_0's l2: 97.4754
[177]	valid_0's l1: 7.27159	valid_0's l2: 97.2872
[178]	valid_0's l1: 7.26002	valid_0's l2: 97.1013
[179]	valid_0's l1: 7.24868	valid_0's l2: 96.9349
[180]	valid_0's l1: 7.23773	valid_0's l2: 96.7806
[181]	valid_0's l1: 7.22645	valid_0's l2: 96.6178
[182]	valid_0's l1: 7.21598	valid_0's l2: 96.4601
[183]	valid_0's l1: 7.2051	valid_0's l2: 96.2972
[184]	valid_0's l1: 7.1941	valid_0's l2: 96.1296
[185]	valid_0's l1: 7.18286	valid_0's l2: 95.9669
[186]	valid_0's l1: 7.17162	valid_0's l2: 95.791
[187]	valid_0's l1: 7.16081	valid_0's l2: 95.6305
[188]	

[333]	valid_0's l1: 6.42573	valid_0's l2: 86.0768
[334]	valid_0's l1: 6.42355	valid_0's l2: 86.0445
[335]	valid_0's l1: 6.42157	valid_0's l2: 86.0199
[336]	valid_0's l1: 6.41922	valid_0's l2: 85.9841
[337]	valid_0's l1: 6.4167	valid_0's l2: 85.9508
[338]	valid_0's l1: 6.41401	valid_0's l2: 85.9203
[339]	valid_0's l1: 6.41242	valid_0's l2: 85.8987
[340]	valid_0's l1: 6.41131	valid_0's l2: 85.886
[341]	valid_0's l1: 6.40994	valid_0's l2: 85.8705
[342]	valid_0's l1: 6.40888	valid_0's l2: 85.857
[343]	valid_0's l1: 6.40709	valid_0's l2: 85.8395
[344]	valid_0's l1: 6.40582	valid_0's l2: 85.8338
[345]	valid_0's l1: 6.40395	valid_0's l2: 85.8188
[346]	valid_0's l1: 6.40241	valid_0's l2: 85.8006
[347]	valid_0's l1: 6.40116	valid_0's l2: 85.782
[348]	valid_0's l1: 6.39958	valid_0's l2: 85.7681
[349]	valid_0's l1: 6.39793	valid_0's l2: 85.7519
[350]	valid_0's l1: 6.39623	valid_0's l2: 85.7278
[351]	valid_0's l1: 6.39531	valid_0's l2: 85.717
[352]	valid_0's l1: 6.3936	valid_0's l2: 85.6946
[353]	

[498]	valid_0's l1: 6.25179	valid_0's l2: 83.6608
[499]	valid_0's l1: 6.25171	valid_0's l2: 83.6536
[500]	valid_0's l1: 6.25133	valid_0's l2: 83.645
[501]	valid_0's l1: 6.25108	valid_0's l2: 83.637
[502]	valid_0's l1: 6.25117	valid_0's l2: 83.6367
[503]	valid_0's l1: 6.25106	valid_0's l2: 83.6293
[504]	valid_0's l1: 6.25152	valid_0's l2: 83.633
[505]	valid_0's l1: 6.25156	valid_0's l2: 83.6261
[506]	valid_0's l1: 6.25064	valid_0's l2: 83.6128
[507]	valid_0's l1: 6.25016	valid_0's l2: 83.6048
[508]	valid_0's l1: 6.24975	valid_0's l2: 83.5974
[509]	valid_0's l1: 6.24828	valid_0's l2: 83.5756
[510]	valid_0's l1: 6.24768	valid_0's l2: 83.5695
[511]	valid_0's l1: 6.24697	valid_0's l2: 83.5531
[512]	valid_0's l1: 6.24638	valid_0's l2: 83.5359
[513]	valid_0's l1: 6.24571	valid_0's l2: 83.5218
[514]	valid_0's l1: 6.24552	valid_0's l2: 83.5169
[515]	valid_0's l1: 6.24491	valid_0's l2: 83.5089
[516]	valid_0's l1: 6.24434	valid_0's l2: 83.4984
[517]	valid_0's l1: 6.24424	valid_0's l2: 83.4958
[51

[64]	valid_0's l1: 9.55179	valid_0's l2: 144.417
[65]	valid_0's l1: 9.51399	valid_0's l2: 143.438
[66]	valid_0's l1: 9.47642	valid_0's l2: 142.48
[67]	valid_0's l1: 9.43889	valid_0's l2: 141.53
[68]	valid_0's l1: 9.40198	valid_0's l2: 140.602
[69]	valid_0's l1: 9.3657	valid_0's l2: 139.693
[70]	valid_0's l1: 9.32966	valid_0's l2: 138.792
[71]	valid_0's l1: 9.29381	valid_0's l2: 137.91
[72]	valid_0's l1: 9.25839	valid_0's l2: 137.027
[73]	valid_0's l1: 9.22486	valid_0's l2: 136.213
[74]	valid_0's l1: 9.19159	valid_0's l2: 135.413
[75]	valid_0's l1: 9.15897	valid_0's l2: 134.624
[76]	valid_0's l1: 9.12704	valid_0's l2: 133.867
[77]	valid_0's l1: 9.09534	valid_0's l2: 133.109
[78]	valid_0's l1: 9.06398	valid_0's l2: 132.373
[79]	valid_0's l1: 9.03108	valid_0's l2: 131.583
[80]	valid_0's l1: 9.00005	valid_0's l2: 130.854
[81]	valid_0's l1: 8.96935	valid_0's l2: 130.137
[82]	valid_0's l1: 8.93806	valid_0's l2: 129.385
[83]	valid_0's l1: 8.90823	valid_0's l2: 128.693
[84]	valid_0's l1: 8.878

...

[479]	valid_0's l1: 6.03263	valid_0's l2: 82.0565
[480]	valid_0's l1: 6.03221	valid_0's l2: 82.0471
[481]	valid_0's l1: 6.03145	valid_0's l2: 82.0362
[482]	valid_0's l1: 6.03106	valid_0's l2: 82.0344
[483]	valid_0's l1: 6.03036	valid_0's l2: 82.0301
[484]	valid_0's l1: 6.02951	valid_0's l2: 82.0142
[485]	valid_0's l1: 6.0289	valid_0's l2: 82.0108
[486]	valid_0's l1: 6.02875	valid_0's l2: 82.0041
[487]	valid_0's l1: 6.02823	valid_0's l2: 82.0046
[488]	valid_0's l1: 6.02767	valid_0's l2: 82.0004
[489]	valid_0's l1: 6.02703	valid_0's l2: 81.9891
[490]	valid_0's l1: 6.02723	valid_0's l2: 81.9928
[491]	valid_0's l1: 6.02743	valid_0's l2: 81.996
[492]	valid_0's l1: 6.02742	valid_0's l2: 82.0057
[493]	valid_0's l1: 6.02705	valid_0's l2: 81.9952
[494]	valid_0's l1: 6.02673	valid_0's l2: 81.9901
[495]	valid_0's l1: 6.02592	valid_0's l2: 81.9854
[496]	valid_0's l1: 6.02564	valid_0's l2: 81.9851
[497]	valid_0's l1: 6.02587	valid_0's l2: 81.9875
[498]	valid_0's l1: 6.02548	valid_0's l2: 81.9858
[4

[45]	valid_0's l1: 10.982	valid_0's l2: 185.322
[46]	valid_0's l1: 10.9258	valid_0's l2: 183.615
[47]	valid_0's l1: 10.8702	valid_0's l2: 181.939
[48]	valid_0's l1: 10.8152	valid_0's l2: 180.31
[49]	valid_0's l1: 10.7602	valid_0's l2: 178.653
[50]	valid_0's l1: 10.7067	valid_0's l2: 177.063
[51]	valid_0's l1: 10.6542	valid_0's l2: 175.537
[52]	valid_0's l1: 10.6023	valid_0's l2: 174.031
[53]	valid_0's l1: 10.5521	valid_0's l2: 172.591
[54]	valid_0's l1: 10.503	valid_0's l2: 171.171
[55]	valid_0's l1: 10.4536	valid_0's l2: 169.76
[56]	valid_0's l1: 10.4042	valid_0's l2: 168.369
[57]	valid_0's l1: 10.3557	valid_0's l2: 167.003
[58]	valid_0's l1: 10.3065	valid_0's l2: 165.606
[59]	valid_0's l1: 10.2594	valid_0's l2: 164.259
[60]	valid_0's l1: 10.211	valid_0's l2: 162.901
[61]	valid_0's l1: 10.164	valid_0's l2: 161.627
[62]	valid_0's l1: 10.1165	valid_0's l2: 160.355
[63]	valid_0's l1: 10.0703	valid_0's l2: 159.108
[64]	valid_0's l1: 10.0253	valid_0's l2: 157.885
[65]	valid_0's l1: 9.98076

[211]	valid_0's l1: 6.99389	valid_0's l2: 93.7169
[212]	valid_0's l1: 6.98675	valid_0's l2: 93.6196
[213]	valid_0's l1: 6.97941	valid_0's l2: 93.524
[214]	valid_0's l1: 6.9727	valid_0's l2: 93.4314
[215]	valid_0's l1: 6.96537	valid_0's l2: 93.326
[216]	valid_0's l1: 6.95912	valid_0's l2: 93.2413
[217]	valid_0's l1: 6.95122	valid_0's l2: 93.1322
[218]	valid_0's l1: 6.9441	valid_0's l2: 93.0297
[219]	valid_0's l1: 6.93745	valid_0's l2: 92.9382
[220]	valid_0's l1: 6.92997	valid_0's l2: 92.8344
[221]	valid_0's l1: 6.92329	valid_0's l2: 92.7406
[222]	valid_0's l1: 6.91714	valid_0's l2: 92.6595
[223]	valid_0's l1: 6.90992	valid_0's l2: 92.5627
[224]	valid_0's l1: 6.90322	valid_0's l2: 92.4749
[225]	valid_0's l1: 6.89606	valid_0's l2: 92.389
[226]	valid_0's l1: 6.88884	valid_0's l2: 92.2893
[227]	valid_0's l1: 6.88177	valid_0's l2: 92.1933
[228]	valid_0's l1: 6.87481	valid_0's l2: 92.0937
[229]	valid_0's l1: 6.86809	valid_0's l2: 92.0008
[230]	valid_0's l1: 6.86174	valid_0's l2: 91.9062
[231]

[376]	valid_0's l1: 6.39124	valid_0's l2: 86.0919
[377]	valid_0's l1: 6.38966	valid_0's l2: 86.0855
[378]	valid_0's l1: 6.38866	valid_0's l2: 86.0781
[379]	valid_0's l1: 6.38634	valid_0's l2: 86.0523
[380]	valid_0's l1: 6.3839	valid_0's l2: 86.0228
[381]	valid_0's l1: 6.38198	valid_0's l2: 86.0012
[382]	valid_0's l1: 6.38132	valid_0's l2: 85.99
[383]	valid_0's l1: 6.3806	valid_0's l2: 85.9868
[384]	valid_0's l1: 6.37893	valid_0's l2: 85.9506
[385]	valid_0's l1: 6.37785	valid_0's l2: 85.9393
[386]	valid_0's l1: 6.37628	valid_0's l2: 85.9177
[387]	valid_0's l1: 6.37468	valid_0's l2: 85.896
[388]	valid_0's l1: 6.3733	valid_0's l2: 85.8791
[389]	valid_0's l1: 6.37161	valid_0's l2: 85.8579
[390]	valid_0's l1: 6.3701	valid_0's l2: 85.8378
[391]	valid_0's l1: 6.36834	valid_0's l2: 85.8181
[392]	valid_0's l1: 6.36697	valid_0's l2: 85.7969
[393]	valid_0's l1: 6.36504	valid_0's l2: 85.7708
[394]	valid_0's l1: 6.36363	valid_0's l2: 85.7459
[395]	valid_0's l1: 6.36138	valid_0's l2: 85.7215
[396]	v

[541]	valid_0's l1: 6.27077	valid_0's l2: 83.8497
[542]	valid_0's l1: 6.27028	valid_0's l2: 83.8337
[543]	valid_0's l1: 6.26981	valid_0's l2: 83.8194
[544]	valid_0's l1: 6.26934	valid_0's l2: 83.8113
[545]	valid_0's l1: 6.26903	valid_0's l2: 83.8041
[546]	valid_0's l1: 6.26866	valid_0's l2: 83.7967
[547]	valid_0's l1: 6.26872	valid_0's l2: 83.797
[548]	valid_0's l1: 6.26888	valid_0's l2: 83.7987
[549]	valid_0's l1: 6.26893	valid_0's l2: 83.7955
[550]	valid_0's l1: 6.26912	valid_0's l2: 83.7893
[551]	valid_0's l1: 6.26889	valid_0's l2: 83.779
[552]	valid_0's l1: 6.26885	valid_0's l2: 83.7742
[553]	valid_0's l1: 6.26882	valid_0's l2: 83.7657
[554]	valid_0's l1: 6.26892	valid_0's l2: 83.7565
[555]	valid_0's l1: 6.26844	valid_0's l2: 83.7387
[556]	valid_0's l1: 6.26834	valid_0's l2: 83.7271
[557]	valid_0's l1: 6.26846	valid_0's l2: 83.7133
[558]	valid_0's l1: 6.26837	valid_0's l2: 83.7082
[559]	valid_0's l1: 6.26739	valid_0's l2: 83.6867
[560]	valid_0's l1: 6.26647	valid_0's l2: 83.6666
[5

In [ ]:
# This builds the CATEGORIZATION labels for a single player at a time.
# Starting with the away team:

away_player_training_list = []
away_player_validation_list = []

for i in range(15):
    player = combined_features[:,(17430+i):(17431+i)]
    player_train = player[:14000,:]
    player_test = player[14000:,:]
    player_train_append = []
    for score in player_train:
        if score > 40:
            player_train_append.append(1)
        else:
            player_train_append.append(0)
    player_test_append = []
    for score in player_test:
        if score > 40:
            player_test_append.append(1)
        else:
            player_test_append.append(0)
    away_player_training_list.append(player_train_append)
    away_player_validation_list.append(player_test_append)

# Then the home team:
home_player_training_list = []
home_player_validation_list = []

for i in range(15):
    player = combined_features[:,17445+i:17446+i]
    player_train = player[:14000,:]
    player_test = player[14000:,:]
    player_train_append = []
    for score in player_train:
        if score > 40:
            player_train_append.append(1)
        else:
            player_train_append.append(0)
    player_test_append = []
    for score in player_test:
        if score > 40:
            player_test_append.append(1)
        else:
            player_test_append.append(0)
    home_player_training_list.append(player_train_append)
    home_player_validation_list.append(player_test_append)

# Cleaning up variables:
combined_features = []
all_labels_away = []
all_labels_home = []
del all_labels_away
del all_labels_home
del combined_features

In [ ]:
# This section trains 15 models on (hopefully) optimized parameters.
away_model_list = ['player1_away', 'player2_away', 'player3_away', 'player4_away', 'player5_away', 
                  'player6_away', 'player7_away', 'player8_away', 'player9_away', 'player10_away', 
                  'player11_away', 'player12_away', 'player13_away', 'player14_away', 'player15_away']

for j in range(15):
    train_y = away_player_training_list[j]
    valid_y = away_player_validation_list[j]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "binary",
        "metric": 'binary_logloss',
        "force_col_wise": True,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": .5,
        "lambda_l2": .5,
        "num_leaves": 1000,
        "bagging_fraction": .9,
        "bagging_freq": 3,
        "min_child_samples": 100,
        'learning_rate': .01,
        'n_estimators': 600,
        'max_depth': 30,
        'max_bin': 400,
    }

    gbm = lgb.train(param, dtrain, valid_sets=[dvalid])
    model_name = away_model_list[j] + '.txt'
    gbm.save_model(model_name)

    # This section trains 15 models on (hopefully) optimized parameters.
home_model_list = ['player1_home', 'player2_home', 'player3_home', 'player4_home', 'player5_home', 
                  'player6_home', 'player7_home', 'player8_home', 'player9_home', 'player10_home', 
                  'player11_home', 'player12_home', 'player13_home', 'player14_home', 'player15_home']

for j in range(15):
    train_y = home_player_training_list[j]
    valid_y = home_player_validation_list[j]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "regression",
        "metric": ['l1','l2'],
        "force_col_wise": True,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": .5,
        "lambda_l2": .5,
        "num_leaves": 1000,
        "bagging_fraction": .9,
        "bagging_freq": 3,
        "min_child_samples": 100,
        'learning_rate': .01,
        'n_estimators': 600,
        'max_depth': 30,
        'max_bin': 400,
    }

    gbm = lgb.train(param, dtrain, valid_sets=[dvalid])
    model_name = home_model_list[j] + '.txt'
    gbm.save_model(model_name)
    print("Completed model: " + str(j+1))


In [25]:
# This section runs an optuna test on the full data set:
def objective(trial):
    train_y = away_player_training_list[0]
    valid_y = away_player_validation_list[0]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

# These are samples of how to assign random variable types:
# "lambda_l1": trial.suggest_float("lambda_l1", 1e-3, 10.0, log=True),
# "num_leaves": trial.suggest_int("num_leaves",500,5000),
# "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
    
    param = {
    "objective": "regression",
    "metric": ['l1','l2'],
    "force_col_wise": True,
    "verbosity": -1,
    "boosting_type": "gbdt",
    "lambda_l1": .5,
    "lambda_l2": .5,
    "num_leaves": trial.suggest_int("num_leaves",100,2000),
    "bagging_fraction": .9,
    "bagging_freq": 3,
    "min_child_samples": 100,
    'learning_rate': .01,
    'n_estimators': 600,
    'max_depth': 30,
    'max_bin': 400,
    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "l2")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
    
    preds = gbm.predict(valid_x)
    accuracy = sklearn.metrics.mean_squared_error(valid_y, preds)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize")
    study.optimize(objective, n_trials=20)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))